In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import ipywidgets as widgets

In [2]:
turmas = pd.read_csv('Dados Turma/turmas-2016.2.csv', sep=';')
disciplinas = pd.read_csv('componentes-curriculares-presenciais.csv', 
                          sep=';', 
                          usecols=["id_componente","codigo","nivel","nome","unidade_responsavel"])
docentes = pd.read_csv('docentes.csv', sep=';')



In [3]:
turmas.rename(columns={'id_componente_curricular': 'id_componente'}, inplace=True)
turmas.id_componente = turmas.id_componente.astype(str)

# Limpando o dataframe disciplinas com linhas desnecessárias
disciplinas.dropna(inplace=True)
disciplinas = disciplinas[~disciplinas.id_componente.duplicated()]




In [4]:
colunas = ["id_turma","id_docente_interno","codigo","nivel_ensino","nome","unidade_responsavel",
          "situacao_turma","capacidade_aluno","descricao_horario"]

merge_turmas_disciplinas = pd.merge(turmas, disciplinas, on='id_componente')[colunas]
merge_turmas_disciplinas.dropna(inplace=True)
merge_turmas_disciplinas.head()

,id_turma,id_docente_interno,codigo,nivel_ensino,nome,unidade_responsavel,situacao_turma,capacidade_aluno,descricao_horario
0,57581492,5753060.0,GPU0009,LATO SENSU,INOVAÇÃO NA GESTÃO DE POLÍTICAS PÚBLICAS,PROGRAMA DE PÓS-GRADUAÇÃO EM GESTÃO PÚBLICA,CONSOLIDADA,50.0,7M2345 35T23456 7T1
1,57583343,5752921.0,MUT311,TÉCNICO,APRECIAÇÃO MUSICAL II,ESCOLA DE MÚSICA,CONSOLIDADA,30.0,2T1234
2,57583344,5752917.0,MUT491,TÉCNICO,ARRANJOS II,ESCOLA DE MÚSICA,CONSOLIDADA,20.0,3T34
3,57583345,23400.0,MUT102,TÉCNICO,CANTO II,ESCOLA DE MÚSICA,CONSOLIDADA,3.0,3T123
4,57583346,35466.0,MUT102,TÉCNICO,CANTO II,ESCOLA DE MÚSICA,CONSOLIDADA,3.0,6T456


In [5]:
# filtrando as turmas com código IMD
turmas_imd = merge_turmas_disciplinas[merge_turmas_disciplinas.codigo.str.contains("IMD")]

# utilizar apenas disciplinas com código "CONSOLIDADA"
turmas_imd = turmas_imd[turmas_imd.situacao_turma == 'CONSOLIDADA']

# Adaptar o nome da coluna id_docente_interno para id_servidor
turmas_imd.rename(columns={'id_docente_interno': 'id_servidor'}, inplace=True)

turmas_imd.capacidade_aluno = turmas_imd.capacidade_aluno.astype(np.int64)

# Adaptar os tipos de dados em turmas_imd para o merge com o dataframe docentes
turmas_imd.id_servidor = turmas_imd.id_servidor.astype(np.int64)
turmas_imd.head()

,id_turma,id_servidor,codigo,nivel_ensino,nome,unidade_responsavel,situacao_turma,capacidade_aluno,descricao_horario
3564,57586177,62353,IMD0029,GRADUAÇÃO,ESTRUTURA DE DADOS BÁSICAS I,INSTITUTO METROPOLE DIGITAL,CONSOLIDADA,60,35M12 (25/07/2016 - 17/12/2016)
3565,57586351,5757975,IMD0029,GRADUAÇÃO,ESTRUTURA DE DADOS BÁSICAS I,INSTITUTO METROPOLE DIGITAL,CONSOLIDADA,60,35T12 (25/07/2016 - 17/12/2016)
3566,57586359,5757905,IMD0029,GRADUAÇÃO,ESTRUTURA DE DADOS BÁSICAS I,INSTITUTO METROPOLE DIGITAL,CONSOLIDADA,60,35N12 (25/07/2016 - 17/12/2016)
3567,57586359,5757071,IMD0029,GRADUAÇÃO,ESTRUTURA DE DADOS BÁSICAS I,INSTITUTO METROPOLE DIGITAL,CONSOLIDADA,60,35N12 (25/07/2016 - 17/12/2016)
3568,57586361,5757883,IMD0029,GRADUAÇÃO,ESTRUTURA DE DADOS BÁSICAS I,INSTITUTO METROPOLE DIGITAL,CONSOLIDADA,40,35N34 (25/07/2016 - 17/12/2016)


In [6]:
# merge os dataframes  turmas_imd e docentes utilizando como base a coluna id_servidor

colunas = ["id_turma", "codigo", "nivel_ensino", "nome_y",
          "nome_x","capacidade_aluno", "descricao_horario"]
turmas_imd_nomesprof = pd.merge(turmas_imd, docentes, on='id_servidor')[colunas]
lista_prof = turmas_imd_nomesprof.nome_y.unique()



In [7]:
lista_prof.sort()
turmas_imd_nomesprof.head()

,id_turma,codigo,nivel_ensino,nome_y,nome_x,capacidade_aluno,descricao_horario
0,57586177,IMD0029,GRADUAÇÃO,SELAN RODRIGUES DOS SANTOS,ESTRUTURA DE DADOS BÁSICAS I,60,35M12 (25/07/2016 - 17/12/2016)
1,57586198,IMD0030,GRADUAÇÃO,SELAN RODRIGUES DOS SANTOS,LINGUAGEM DE PROGRAMAÇÃO I,30,35M34 (25/07/2016 - 17/12/2016)
2,57586359,IMD0029,GRADUAÇÃO,CESAR RENNO COSTA,ESTRUTURA DE DADOS BÁSICAS I,60,35N12 (25/07/2016 - 17/12/2016)
3,57586373,IMD0030,GRADUAÇÃO,CESAR RENNO COSTA,LINGUAGEM DE PROGRAMAÇÃO I,32,35N34 (25/07/2016 - 17/12/2016)
4,57586359,IMD0029,GRADUAÇÃO,JORGE ESTEFANO SANTANA DE SOUZA,ESTRUTURA DE DADOS BÁSICAS I,60,35N12 (25/07/2016 - 17/12/2016)


In [8]:
#Essa celula pega a avaliação dos professores do imd
avaliacao = pd.read_csv("avaliacaodocencia.csv", sep=';')
av_imd = avaliacao[avaliacao['nome_docente'].isin(lista_prof)]
av_imd.head()

,id_docente,nome_docente,id_turma,ano,periodo,qtd_discentes,postura_profissional_media,postura_profissional_DP,atuacao_profissional_media,atuacao_profissional_DP,autoavaliacao_aluno_media,autoavaliacao_aluno_DP
12856,24853,ANNE MAGALY DE PAULA CANUTO,1208019,2013,2,33,9.43,0.91,9.27,1.21,8.64,2.16
12857,24853,ANNE MAGALY DE PAULA CANUTO,1211641,2013,2,12,9.42,0.97,8.72,1.80,7.90,2.55
12858,24853,ANNE MAGALY DE PAULA CANUTO,1217954,2014,1,19,9.23,1.81,7.63,2.94,7.24,3.38
12859,24853,ANNE MAGALY DE PAULA CANUTO,1228386,2014,2,17,9.50,0.83,7.67,2.71,8.31,2.50
12860,24853,ANNE MAGALY DE PAULA CANUTO,1239412,2015,1,25,9.25,1.27,8.78,1.52,7.87,2.54


In [9]:
# merge os dataframes  av_imd e turmas_imd utilizando como base a coluna id_turma

av_imd_nomesprof = pd.merge(av_imd, turmas_imd, on='id_turma')
av_imd_nomesprof.head()

,id_docente,nome_docente,id_turma,ano,periodo,qtd_discentes,postura_profissional_media,postura_profissional_DP,atuacao_profissional_media,atuacao_profissional_DP,autoavaliacao_aluno_media,autoavaliacao_aluno_DP,id_servidor,codigo,nivel_ensino,nome,unidade_responsavel,situacao_turma,capacidade_aluno,descricao_horario
0,24853,ANNE MAGALY DE PAULA CANUTO,57586394,2016,2,12,10.00,0.00,9.47,1.14,7.89,3.24,24853,IMD0033,GRADUAÇÃO,PROBABILIDADE,INSTITUTO METROPOLE DIGITAL,CONSOLIDADA,60,24M12 (25/07/2016 - 17/12/2016)
1,24949,ANDRE MAURICIO CUNHA CAMPOS,57586250,2016,2,16,9.46,0.83,9.05,1.48,8.47,2.13,24949,IMD0012.0,GRADUAÇÃO,INTRODUÇÃO ÀS TÉCNICAS DE PROGRAMAÇÃO,INSTITUTO METROPOLE DIGITAL,CONSOLIDADA,60,35M34 (25/07/2016 - 17/12/2016)
2,24949,ANDRE MAURICIO CUNHA CAMPOS,57586259,2016,2,32,9.25,1.49,8.16,2.83,8.66,2.36,24949,IMD0012.1,GRADUAÇÃO,PRÁTICAS DE TÉCNICAS DE PROGRAMAÇÃO,INSTITUTO METROPOLE DIGITAL,CONSOLIDADA,40,3M56 (25/07/2016 - 17/12/2016)
3,24949,ANDRE MAURICIO CUNHA CAMPOS,57586277,2016,2,16,9.50,0.80,8.75,1.86,8.09,2.79,24949,IMD0012.1,GRADUAÇÃO,PRÁTICAS DE TÉCNICAS DE PROGRAMAÇÃO,INSTITUTO METROPOLE DIGITAL,CONSOLIDADA,40,5M56 (25/07/2016 - 17/12/2016)
4,25868,UMBERTO SOUZA DA COSTA,57586366,2016,2,23,9.88,0.33,8.38,2.35,8.57,1.99,25868,IMD0030,GRADUAÇÃO,LINGUAGEM DE PROGRAMAÇÃO I,INSTITUTO METROPOLE DIGITAL,CONSOLIDADA,30,35T34 (25/07/2016 - 17/12/2016)


In [10]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual

def evolucao(x):
    dados_prof = av_imd_nomesprof.loc[av_imd_nomesprof['nome_docente'] == x]
    return (dados_prof)

interact(evolucao, x=lista_prof,);

interactive(children=(Dropdown(description='x', options=('ADJA FERREIRA DE ANDRADE', 'ALLAN DE MEDEIROS MARTINS', 'ALUIZIO FERREIRA DA ROCHA NETO', 'ALYSON MATHEUS DE CARVALHO SOUZA', 'ANDRE LUIZ DA SILVA SOLINO', 'ANDRE MAURICIO CUNHA CAMPOS', 'ANNE MAGALY DE PAULA CANUTO', 'ANTONIO IGOR SILVA DE OLIVEIRA', 'ANTONIO WALLACE ANTUNES SOARES', 'ATHANASIOS TSOUANAS', 'AUGUSTO JOSE VENANCIO NETO', 'BRUNO SANTANA DA SILVA', 'CARLOS AUGUSTO PROLO', 'CESAR RENNO COSTA', 'CHARLES ANDRYE GALVAO MADEIRA', 'DANILO CURVELO DE SOUZA', 'DENNYS LEITE MAIA', 'DIOMADSON RODRIGUES BELFORT', 'EDGARD DE FARIA CORREA', 'EDUARDO NOGUEIRA CUNHA', 'EIJI ADACHI MEDEIROS BARBOSA', 'EVERTON RANIELLY DE SOUSA CAVALCANTE', 'GIBEON SOARES DE AQUINO JUNIOR', 'GUSTAVO GIRAO BARRETO DA SILVA', 'IRIS LINHARES PIMENTA', 'ISAAC FRANCO FERNANDES', 'ISABEL DILLMANN NUNES', 'ITAMIR DE MORAIS BARROCA FILHO', 'IVANOVITCH MEDEIROS DANTAS DA SILVA', 'JOAO CARLOS XAVIER JUNIOR', 'JOAO MARCOS DE ALMEIDA', 'JORGE ESTEFANO SANTANA DE SOUZA', 'JULIO CESAR PAULINO DE MELO', 'LORENA AZEVEDO DE SOUSA', 'LOURENA KARIN DE MEDEIROS ROCHA', 'LUCELIO DANTAS DE AQUINO', 'MARCIA JACYNTHA NUNES RODRIGUES LUCENA', 'MONICA MAGALHAES PEREIRA', 'NELSON ION DE OLIVEIRA', 'RAFAELA HORACINA SILVA ROCHA SOARES', 'SAMYR SILVA BEZERRA JACOME', 'SELAN RODRIGUES DOS SANTOS', 'SILVIA MARIA DINIZ MONTEIRO MAIA', 'SILVIO COSTA SAMPAIO', 'UMBERTO RIVIECCIO', 'UMBERTO SOUZA DA COSTA', 'VICENTE ANGELO DE SOUSA JUNIOR', 'WELLINGTON SILVA DE SOUZA', 'WILLIAM BRENNO DOS SANTOS OLIVEIRA'), value='ADJA FERREIRA DE ANDRADE'), Output()), _dom_classes=('widget-interact',))

In [11]:
% matplotlib inline
from ipywidgets import interactive
import matplotlib.pyplot as plt
import numpy as np

def evolucaoGrafico(x):
    dados_prof = av_imd_nomesprof.loc[av_imd_nomesprof['nome_docente'] == x]
    plt.barh(dados_prof['nome'], dados_prof['postura_profissional_media'], 0.3)
    plt.xlabel("Medias")
    plt.ylabel("Disciplinas")
    plt.title("Notas das disciplinas no semestre de 2016.2")
    plt.show()
    
interactive_plot = interact(evolucaoGrafico, x=lista_prof,)
interactive_plot

interactive(children=(Dropdown(description='x', options=('ADJA FERREIRA DE ANDRADE', 'ALLAN DE MEDEIROS MARTINS', 'ALUIZIO FERREIRA DA ROCHA NETO', 'ALYSON MATHEUS DE CARVALHO SOUZA', 'ANDRE LUIZ DA SILVA SOLINO', 'ANDRE MAURICIO CUNHA CAMPOS', 'ANNE MAGALY DE PAULA CANUTO', 'ANTONIO IGOR SILVA DE OLIVEIRA', 'ANTONIO WALLACE ANTUNES SOARES', 'ATHANASIOS TSOUANAS', 'AUGUSTO JOSE VENANCIO NETO', 'BRUNO SANTANA DA SILVA', 'CARLOS AUGUSTO PROLO', 'CESAR RENNO COSTA', 'CHARLES ANDRYE GALVAO MADEIRA', 'DANILO CURVELO DE SOUZA', 'DENNYS LEITE MAIA', 'DIOMADSON RODRIGUES BELFORT', 'EDGARD DE FARIA CORREA', 'EDUARDO NOGUEIRA CUNHA', 'EIJI ADACHI MEDEIROS BARBOSA', 'EVERTON RANIELLY DE SOUSA CAVALCANTE', 'GIBEON SOARES DE AQUINO JUNIOR', 'GUSTAVO GIRAO BARRETO DA SILVA', 'IRIS LINHARES PIMENTA', 'ISAAC FRANCO FERNANDES', 'ISABEL DILLMANN NUNES', 'ITAMIR DE MORAIS BARROCA FILHO', 'IVANOVITCH MEDEIROS DANTAS DA SILVA', 'JOAO CARLOS XAVIER JUNIOR', 'JOAO MARCOS DE ALMEIDA', 'JORGE ESTEFANO SANTANA DE SOUZA', 'JULIO CESAR PAULINO DE MELO', 'LORENA AZEVEDO DE SOUSA', 'LOURENA KARIN DE MEDEIROS ROCHA', 'LUCELIO DANTAS DE AQUINO', 'MARCIA JACYNTHA NUNES RODRIGUES LUCENA', 'MONICA MAGALHAES PEREIRA', 'NELSON ION DE OLIVEIRA', 'RAFAELA HORACINA SILVA ROCHA SOARES', 'SAMYR SILVA BEZERRA JACOME', 'SELAN RODRIGUES DOS SANTOS', 'SILVIA MARIA DINIZ MONTEIRO MAIA', 'SILVIO COSTA SAMPAIO', 'UMBERTO RIVIECCIO', 'UMBERTO SOUZA DA COSTA', 'VICENTE ANGELO DE SOUSA JUNIOR', 'WELLINGTON SILVA DE SOUZA', 'WILLIAM BRENNO DOS SANTOS OLIVEIRA'), value='ADJA FERREIRA DE ANDRADE'), Output()), _dom_classes=('widget-interact',))

<function __main__.evolucaoGrafico>

In [12]:
turmas2 = pd.read_csv('Dados Turma/turmas-2016.1.csv', sep=';')
turmas2.head()

,id_turma,codigo_turma,id_docente_interno,id_docente_externo,observacao,id_componente_curricular,nivel_ensino,campus_turma,local,ano,...,tipo,distancia,data_consolidacao,agrupadora,id_turma_agrupadora,qtd_aulas_lancadas,situacao_turma,convenio,modalidade_participantes,Unnamed: 25
0,57562257,01,NaN,109881493.0,NaN,56847,LATO SENSU,NaN,NaN,2016,...,REGULAR,t,2016-05-30 13:42:46.443,f,NaN,0.0,CONSOLIDADA,NaN,A Distância,NaN
1,57562261,02,5752206.0,NaN,NaN,56847,LATO SENSU,NaN,NaN,2016,...,REGULAR,t,2016-05-25 10:05:57.996,f,NaN,0.0,CONSOLIDADA,NaN,A Distância,NaN
2,57562261,02,NaN,109881524.0,NaN,56847,LATO SENSU,NaN,NaN,2016,...,REGULAR,t,2016-05-25 10:05:57.996,f,NaN,0.0,CONSOLIDADA,NaN,A Distância,NaN
3,57562265,03,5752206.0,NaN,NaN,56847,LATO SENSU,NaN,NaN,2016,...,REGULAR,t,2016-06-13 08:59:37.463,f,NaN,0.0,CONSOLIDADA,NaN,A Distância,NaN
4,57562265,03,NaN,110569107.0,NaN,56847,LATO SENSU,NaN,NaN,2016,...,REGULAR,t,2016-06-13 08:59:37.463,f,NaN,0.0,CONSOLIDADA,NaN,A Distância,NaN


In [13]:
turmas2.rename(columns={'id_componente_curricular': 'id_componente'}, inplace=True)
turmas2.id_componente = turmas.id_componente.astype(str)

In [16]:
colunas = ["id_turma","id_docente_interno","codigo","nivel_ensino","nome","unidade_responsavel",
          "situacao_turma","capacidade_aluno","descricao_horario"]

merge_turmas_disciplinas2 = pd.merge(turmas2, disciplinas, on='id_componente')[colunas]
merge_turmas_disciplinas2.dropna(inplace=True)
merge_turmas_disciplinas2.head()

,id_turma,id_docente_interno,codigo,nivel_ensino,nome,unidade_responsavel,situacao_turma,capacidade_aluno,descricao_horario
11,57573186,23400.0,MUT497,TÉCNICO,RECITAL,ESCOLA DE MÚSICA,CONSOLIDADA,3.0,3T123
12,57573187,26652.0,MUT497,TÉCNICO,RECITAL,ESCOLA DE MÚSICA,EXCLUÍDA,50.0,4T1234
13,57573188,22886.0,MUT497,TÉCNICO,RECITAL,ESCOLA DE MÚSICA,CONSOLIDADA,3.0,4T123
14,57573189,22886.0,MUT497,TÉCNICO,RECITAL,ESCOLA DE MÚSICA,CONSOLIDADA,3.0,4T456
15,57573271,5751559.0,MUT497,TÉCNICO,RECITAL,ESCOLA DE MÚSICA,EXCLUÍDA,3.0,3T456


In [17]:
# filtrando as turmas com código IMD
turmas_imd2 = merge_turmas_disciplinas2[merge_turmas_disciplinas2.codigo.str.contains("IMD")]

# utilizar apenas disciplinas com código "CONSOLIDADA"
turmas_imd2 = turmas_imd2[turmas_imd2.situacao_turma == 'CONSOLIDADA']

# Adaptar o nome da coluna id_docente_interno para id_servidor
turmas_imd2.rename(columns={'id_docente_interno': 'id_servidor'}, inplace=True)

turmas_imd2.capacidade_aluno = turmas_imd2.capacidade_aluno.astype(np.int64)

# Adaptar os tipos de dados em turmas_imd para o merge com o dataframe docentes
turmas_imd2.id_servidor = turmas_imd2.id_servidor.astype(np.int64)
turmas_imd2.head()

,id_turma,id_servidor,codigo,nivel_ensino,nome,unidade_responsavel,situacao_turma,capacidade_aluno,descricao_horario
3564,57575347,5756239,IMD0029,GRADUAÇÃO,ESTRUTURA DE DADOS BÁSICAS I,INSTITUTO METROPOLE DIGITAL,CONSOLIDADA,30,35M234 (05/05/2016 - 18/06/2016)
3565,57575484,22907,IMD0029,GRADUAÇÃO,ESTRUTURA DE DADOS BÁSICAS I,INSTITUTO METROPOLE DIGITAL,CONSOLIDADA,6,24M12 (01/02/2016 - 18/06/2016)
3566,57575486,2300648,IMD0029,GRADUAÇÃO,ESTRUTURA DE DADOS BÁSICAS I,INSTITUTO METROPOLE DIGITAL,CONSOLIDADA,6,24M12 (01/02/2016 - 18/06/2016)
3567,57575486,22869,IMD0029,GRADUAÇÃO,ESTRUTURA DE DADOS BÁSICAS I,INSTITUTO METROPOLE DIGITAL,CONSOLIDADA,6,24M12 (01/02/2016 - 18/06/2016)
3568,57575486,37281,IMD0029,GRADUAÇÃO,ESTRUTURA DE DADOS BÁSICAS I,INSTITUTO METROPOLE DIGITAL,CONSOLIDADA,6,24M12 (01/02/2016 - 18/06/2016)


In [18]:
# merge os dataframes  turmas_imd e docentes utilizando como base a coluna id_servidor

colunas = ["id_turma", "codigo", "nivel_ensino", "nome_y",
          "nome_x","capacidade_aluno", "descricao_horario"]
turmas_imd_nomesprof2 = pd.merge(turmas_imd2, docentes, on='id_servidor')[colunas]
lista_prof2 = turmas_imd_nomesprof2.nome_y.unique()

In [19]:
lista_prof2.sort()
turmas_imd_nomesprof2.head()

,id_turma,codigo,nivel_ensino,nome_y,nome_x,capacidade_aluno,descricao_horario
0,57575347,IMD0029,GRADUAÇÃO,DIEGO ANDRES LAPLAGNE,ESTRUTURA DE DADOS BÁSICAS I,30,35M234 (05/05/2016 - 18/06/2016)
1,57575484,IMD0029,GRADUAÇÃO,GILMAR AMORIM DE SOUSA,ESTRUTURA DE DADOS BÁSICAS I,6,24M12 (01/02/2016 - 18/06/2016)
2,57575486,IMD0030,GRADUAÇÃO,GILMAR AMORIM DE SOUSA,LINGUAGEM DE PROGRAMAÇÃO I,6,24M12 (01/02/2016 - 18/06/2016)
3,57575488,IMD0030,GRADUAÇÃO,GILMAR AMORIM DE SOUSA,LINGUAGEM DE PROGRAMAÇÃO I,6,24M12 (01/02/2016 - 18/06/2016)
4,57575486,IMD0029,GRADUAÇÃO,ENIO CAMPOS AMICO,ESTRUTURA DE DADOS BÁSICAS I,6,24M12 (01/02/2016 - 18/06/2016)


In [20]:
#Essa celula pega a avaliação dos professores do imd
avaliacao = pd.read_csv("avaliacaodocencia.csv", sep=';')
av_imd = avaliacao[avaliacao['nome_docente'].isin(lista_prof2)]
av_imd.head()

,id_docente,nome_docente,id_turma,ano,periodo,qtd_discentes,postura_profissional_media,postura_profissional_DP,atuacao_profissional_media,atuacao_profissional_DP,autoavaliacao_aluno_media,autoavaliacao_aluno_DP
4310,19582,JOSE VERISSIMO FERNANDES,1209382,2013,2,33,9.08,1.55,8.24,2.13,8.67,1.97
4311,19582,JOSE VERISSIMO FERNANDES,1210724,2013,2,20,9.42,1.46,8.29,1.86,8.54,2.32
4312,19582,JOSE VERISSIMO FERNANDES,1210734,2013,2,20,9.75,0.63,8.96,1.40,8.88,1.77
4313,19582,JOSE VERISSIMO FERNANDES,1211258,2013,2,50,9.88,0.55,9.65,0.92,9.29,1.69
4314,19582,JOSE VERISSIMO FERNANDES,1219949,2014,1,18,9.91,0.29,7.99,2.44,8.60,2.36


In [21]:
av_imd_nomesprof2 = pd.merge(av_imd, turmas_imd2, on='id_turma')
av_imd_nomesprof2.head()

,id_docente,nome_docente,id_turma,ano,periodo,qtd_discentes,postura_profissional_media,postura_profissional_DP,atuacao_profissional_media,atuacao_profissional_DP,autoavaliacao_aluno_media,autoavaliacao_aluno_DP,id_servidor,codigo,nivel_ensino,nome,unidade_responsavel,situacao_turma,capacidade_aluno,descricao_horario
0,19582,JOSE VERISSIMO FERNANDES,57578146,2016,1,22,9.83,0.38,8.97,1.72,9.09,1.60,19582,IMD0703,GRADUAÇÃO,SEGURANÇA DE REDES,INSTITUTO METROPOLE DIGITAL,CONSOLIDADA,35,4N34 5N12
1,20854,MARIA TEREZA BARRETO DE OLIVEIRA,57575470,2016,1,14,9.77,0.53,9.63,0.81,9.32,1.45,20854,IMD0034,GRADUAÇÃO,VETORES E GEOMETRIA ANALÍTICA,INSTITUTO METROPOLE DIGITAL,CONSOLIDADA,50,4M123 5M34
2,21470,REGINA DE FATIMA DOS SANTOS BRAZ,57575439,2016,1,51,9.81,0.69,8.54,2.32,9.04,1.97,21470,IMD0012.0,GRADUAÇÃO,INTRODUÇÃO ÀS TÉCNICAS DE PROGRAMAÇÃO,INSTITUTO METROPOLE DIGITAL,CONSOLIDADA,60,3M234
3,21990,DEUSIMAR FREIRE BRASIL,57575400,2016,1,11,9.86,0.36,9.55,0.83,9.39,1.22,21990,IMD0017,GRADUAÇÃO,PRÁTICAS DE LEITURA E ESCRITA EM PORTUGUÊS I,INSTITUTO METROPOLE DIGITAL,CONSOLIDADA,20,46M23
4,22869,SAMIR ASSI JOAO,57575471,2016,1,45,9.33,1.65,8.63,2.43,9.64,0.94,5754345,IMD0034,GRADUAÇÃO,VETORES E GEOMETRIA ANALÍTICA,INSTITUTO METROPOLE DIGITAL,CONSOLIDADA,56,35M12


In [22]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual

def evolucao(x):
    dados_prof = av_imd_nomesprof2.loc[av_imd_nomesprof2['nome_docente'] == x]
    return (dados_prof)

interact(evolucao, x=lista_prof2,);

interactive(children=(Dropdown(description='x', options=('ANA CRISTINA ARAUJO DE ANDRADE GALVAO', 'ANDRE FERRER PINTO MARTINS', 'ANDRE LUIS PINTO SOARES', 'ANNE CHRISTINE DAMASIO', 'AUZELIVIA PASTORA REGO MEDEIROS FALCAO', 'CARLA ALMEIDA VIVACQUA', 'CARLOS ALBERTO ALMEIDA DE ARAUJO', 'CAROLINE ADDISON CARVALHO XAVIER DE MEDEIROS', 'CLEONICE ANDREA ALVES CAVALCANTE', 'DANIELLE SOARES BEZERRA', 'DEUSIMAR FREIRE BRASIL', 'DIEGO ANDRES LAPLAGNE', 'ENIO CAMPOS AMICO', 'EUZEBIO GUIMARAES BARBOSA', 'EZEQUIAS OLIVEIRA LIRA', 'FABIANA BARBOSA GONCALVES', 'FABIANA CRISTINA MENDONCA DE ARAUJO', 'FABIANA LIMA BEZERRA', 'FABRICIO PITOMBO LEITE', 'FERNANDA DA FONSECA FREITAS', 'FLAVIO EDUARDO FALCAO LEMOS', 'GERLANE COELHO BERNARDO GUERRA', 'GILMAR AMORIM DE SOUSA', 'JACILEIDE GUIMARAES', 'JORGE EDUARDO LINS OLIVEIRA', 'JOSE BRUNO DE ALMEIDA', 'JOSE VERISSIMO FERNANDES', 'KATYA ANAYA JACINTO', 'KELLY SAMARA DE LIRA MOTA', 'LINDEMBERG DE JESUS NOGUEIRA DUARTE', 'LUCIANNA PEREIRA DA MOTTA PIRES CORREIA', 'MAGNA ANGELICA DOS SANTOS BEZERRA SOUSA', 'MAGNOLIA FERNANDES FLORENCIO DE ARAUJO', 'MARCELO HENRIQUE NEVES PEREIRA', 'MARCIA MARIA DE OLIVEIRA BEZERRA', 'MARCIO ROMEU RIBAS DE OLIVEIRA', 'MARCIO VIEIRA DA SILVA', 'MARCOS ADLLER DE ALMEIDA NASCIMENTO', 'MARGARETH DE VASCONCELOS MONTEIRO', 'MARIA INES SUCUPIRA STAMATTO', 'MARIA TEREZA BARRETO DE OLIVEIRA', 'MARILENE BIZERRA DA COSTA', 'MAURICIO GALVAO PEREIRA', 'MOISES DOMINGOS SOBRINHO', 'ORIONE DANTAS DE MEDEIROS', 'PAULA REJANE FERNANDES', 'PAULO ROBERTO DE ALBUQUERQUE', 'PIO MARINHEIRO DE SOUZA NETO', 'REGINA DE FATIMA DOS SANTOS BRAZ', 'RENATA VEIGA ANDERSEN CAVALCANTI', 'RENATO DE ALBUQUERQUE MEDEIROS', 'REYNALDO MARTINS E QUININO', 'RICARDO WAGNER DE ARAUJO', 'ROSANGELA DOS SANTOS ALVES PEQUENO', 'RUTACIO DE OLIVEIRA COSTA', 'SAMIR ASSI JOAO', 'SANDRA MICHELLE BESSA DE ANDRADE FERNANDES', 'SERGIO JOSE GONCALVES E SILVA', 'SERGIO RAMIRO RIVERO GUARDIA', 'SUELENE SUASSUNA SILVESTRE DE ALENCAR', 'SUZIANNE RUTH HOSANAH LIMA', 'TIRZAH BRAZ PETTA LAJUS', 'VANESSA TEIXEIRA DE LIMA OLIVEIRA', 'VIRGINIA MARIA CAVALARI HENRIQUES', 'WILACI EUTROPIO FERNANDES JUNIOR'), value='ANA CRISTINA ARAUJO DE ANDRADE GALVAO'), Output()), _dom_classes=('widget-interact',))

In [24]:
% matplotlib inline
from ipywidgets import interactive
import matplotlib.pyplot as plt
import numpy as np

def evolucaoGrafico(x):
    dados_prof2 = av_imd_nomesprof2.loc[av_imd_nomesprof2['nome_docente'] == x]
    plt.barh(dados_prof2['nome'], dados_prof2['postura_profissional_media'], 0.3)
    plt.xlabel("Notas")
    plt.ylabel("Disciplinas")
    plt.title("Notas das disciplinas no semestre de 2016.2")
    plt.show()
    
interactive_plot = interact(evolucaoGrafico, x=lista_prof2,)
interactive_plot

interactive(children=(Dropdown(description='x', options=('ANA CRISTINA ARAUJO DE ANDRADE GALVAO', 'ANDRE FERRER PINTO MARTINS', 'ANDRE LUIS PINTO SOARES', 'ANNE CHRISTINE DAMASIO', 'AUZELIVIA PASTORA REGO MEDEIROS FALCAO', 'CARLA ALMEIDA VIVACQUA', 'CARLOS ALBERTO ALMEIDA DE ARAUJO', 'CAROLINE ADDISON CARVALHO XAVIER DE MEDEIROS', 'CLEONICE ANDREA ALVES CAVALCANTE', 'DANIELLE SOARES BEZERRA', 'DEUSIMAR FREIRE BRASIL', 'DIEGO ANDRES LAPLAGNE', 'ENIO CAMPOS AMICO', 'EUZEBIO GUIMARAES BARBOSA', 'EZEQUIAS OLIVEIRA LIRA', 'FABIANA BARBOSA GONCALVES', 'FABIANA CRISTINA MENDONCA DE ARAUJO', 'FABIANA LIMA BEZERRA', 'FABRICIO PITOMBO LEITE', 'FERNANDA DA FONSECA FREITAS', 'FLAVIO EDUARDO FALCAO LEMOS', 'GERLANE COELHO BERNARDO GUERRA', 'GILMAR AMORIM DE SOUSA', 'JACILEIDE GUIMARAES', 'JORGE EDUARDO LINS OLIVEIRA', 'JOSE BRUNO DE ALMEIDA', 'JOSE VERISSIMO FERNANDES', 'KATYA ANAYA JACINTO', 'KELLY SAMARA DE LIRA MOTA', 'LINDEMBERG DE JESUS NOGUEIRA DUARTE', 'LUCIANNA PEREIRA DA MOTTA PIRES CORREIA', 'MAGNA ANGELICA DOS SANTOS BEZERRA SOUSA', 'MAGNOLIA FERNANDES FLORENCIO DE ARAUJO', 'MARCELO HENRIQUE NEVES PEREIRA', 'MARCIA MARIA DE OLIVEIRA BEZERRA', 'MARCIO ROMEU RIBAS DE OLIVEIRA', 'MARCIO VIEIRA DA SILVA', 'MARCOS ADLLER DE ALMEIDA NASCIMENTO', 'MARGARETH DE VASCONCELOS MONTEIRO', 'MARIA INES SUCUPIRA STAMATTO', 'MARIA TEREZA BARRETO DE OLIVEIRA', 'MARILENE BIZERRA DA COSTA', 'MAURICIO GALVAO PEREIRA', 'MOISES DOMINGOS SOBRINHO', 'ORIONE DANTAS DE MEDEIROS', 'PAULA REJANE FERNANDES', 'PAULO ROBERTO DE ALBUQUERQUE', 'PIO MARINHEIRO DE SOUZA NETO', 'REGINA DE FATIMA DOS SANTOS BRAZ', 'RENATA VEIGA ANDERSEN CAVALCANTI', 'RENATO DE ALBUQUERQUE MEDEIROS', 'REYNALDO MARTINS E QUININO', 'RICARDO WAGNER DE ARAUJO', 'ROSANGELA DOS SANTOS ALVES PEQUENO', 'RUTACIO DE OLIVEIRA COSTA', 'SAMIR ASSI JOAO', 'SANDRA MICHELLE BESSA DE ANDRADE FERNANDES', 'SERGIO JOSE GONCALVES E SILVA', 'SERGIO RAMIRO RIVERO GUARDIA', 'SUELENE SUASSUNA SILVESTRE DE ALENCAR', 'SUZIANNE RUTH HOSANAH LIMA', 'TIRZAH BRAZ PETTA LAJUS', 'VANESSA TEIXEIRA DE LIMA OLIVEIRA', 'VIRGINIA MARIA CAVALARI HENRIQUES', 'WILACI EUTROPIO FERNANDES JUNIOR'), value='ANA CRISTINA ARAUJO DE ANDRADE GALVAO'), Output()), _dom_classes=('widget-interact',))

<function __main__.evolucaoGrafico>

In [75]:
turmas3 = pd.read_csv('Dados Turma/turmas-2015.2.csv', sep=';')
turmas4 = pd.read_csv('Dados Turma/turmas-2015.1.csv', sep=';')
turmas5 = pd.read_csv('Dados Turma/turmas-2014.2.csv', sep=';')
turmas6 = pd.read_csv('Dados Turma/turmas-2014.1.csv', sep=';')

In [76]:
turmas3.rename(columns={'id_componente_curricular': 'id_componente'}, inplace=True)
turmas3.id_componente = turmas.id_componente.astype(str)

turmas4.rename(columns={'id_componente_curricular': 'id_componente'}, inplace=True)
turmas4.id_componente = turmas.id_componente.astype(str)

turmas5.rename(columns={'id_componente_curricular': 'id_componente'}, inplace=True)
turmas5.id_componente = turmas.id_componente.astype(str)

turmas6.rename(columns={'id_componente_curricular': 'id_componente'}, inplace=True)
turmas6.id_componente = turmas.id_componente.astype(str)

In [78]:
colunas = ["id_turma","id_docente_interno","codigo","nivel_ensino","nome","unidade_responsavel",
          "situacao_turma","capacidade_aluno","descricao_horario"]

merge_turmas_disciplinas3 = pd.merge(turmas3, disciplinas, on='id_componente')[colunas]
merge_turmas_disciplinas3.dropna(inplace=True)
merge_turmas_disciplinas3.head()

,id_turma,id_docente_interno,codigo,nivel_ensino,nome,unidade_responsavel,situacao_turma,capacidade_aluno,descricao_horario
0,57560974,5755580.0,GPU0009,LATO SENSU,INOVAÇÃO NA GESTÃO DE POLÍTICAS PÚBLICAS,PROGRAMA DE PÓS-GRADUAÇÃO EM GESTÃO PÚBLICA,CONSOLIDADA,60.0,7M12345 7T12345
1,57560975,5756402.0,MUT311,LATO SENSU,APRECIAÇÃO MUSICAL II,ESCOLA DE MÚSICA,CONSOLIDADA,60.0,7M12345 7T12345
2,57560976,22193.0,MUT491,LATO SENSU,ARRANJOS II,ESCOLA DE MÚSICA,CONSOLIDADA,60.0,7M12345 7T12345
3,57560977,5755386.0,MUT102,LATO SENSU,CANTO II,ESCOLA DE MÚSICA,CONSOLIDADA,60.0,7M12345 7T12345
4,57560982,5755973.0,MUT102,LATO SENSU,CANTO II,ESCOLA DE MÚSICA,CONSOLIDADA,60.0,7M12345 7T12345


In [79]:
colunas = ["id_turma","id_docente_interno","codigo","nivel_ensino","nome","unidade_responsavel",
          "situacao_turma","capacidade_aluno","descricao_horario"]

merge_turmas_disciplinas4 = pd.merge(turmas4, disciplinas, on='id_componente')[colunas]
merge_turmas_disciplinas4.dropna(inplace=True)
merge_turmas_disciplinas4.head()

,id_turma,id_docente_interno,codigo,nivel_ensino,nome,unidade_responsavel,situacao_turma,capacidade_aluno,descricao_horario
0,1217099,18225.0,GPU0009,LATO SENSU,INOVAÇÃO NA GESTÃO DE POLÍTICAS PÚBLICAS,PROGRAMA DE PÓS-GRADUAÇÃO EM GESTÃO PÚBLICA,CONSOLIDADA,26.0,23456T234
1,1217100,18695.0,MUT311,LATO SENSU,APRECIAÇÃO MUSICAL II,ESCOLA DE MÚSICA,CONSOLIDADA,26.0,345T2345 6T234 (14/04/2015 - 17/04/2015)
2,1217101,5753068.0,MUT491,LATO SENSU,ARRANJOS II,ESCOLA DE MÚSICA,CONSOLIDADA,26.0,23456T123456
5,1237217,26152.0,MUT102,LATO SENSU,CANTO II,ESCOLA DE MÚSICA,CONSOLIDADA,12.0,6T2345
6,1237222,5753866.0,MUT102,LATO SENSU,CANTO II,ESCOLA DE MÚSICA,CONSOLIDADA,12.0,6N234


In [80]:
colunas = ["id_turma","id_docente_interno","codigo","nivel_ensino","nome","unidade_responsavel",
          "situacao_turma","capacidade_aluno","descricao_horario"]

merge_turmas_disciplinas5 = pd.merge(turmas5, disciplinas, on='id_componente')[colunas]
merge_turmas_disciplinas5.dropna(inplace=True)
merge_turmas_disciplinas5.head()

,id_turma,id_docente_interno,codigo,nivel_ensino,nome,unidade_responsavel,situacao_turma,capacidade_aluno,descricao_horario
0,1207250,5755580,GPU0009,LATO SENSU,INOVAÇÃO NA GESTÃO DE POLÍTICAS PÚBLICAS,PROGRAMA DE PÓS-GRADUAÇÃO EM GESTÃO PÚBLICA,CONSOLIDADA,33,7M2345 7T3456
1,1213722,23859,MUT311,LATO SENSU,APRECIAÇÃO MUSICAL II,ESCOLA DE MÚSICA,CONSOLIDADA,33,7M12345 7T23456 (20/09/2014 - 20/12/2014)
2,1217095,3029296,MUT491,LATO SENSU,ARRANJOS II,ESCOLA DE MÚSICA,CONSOLIDADA,26,23456T123456 (14/07/2014 - 18/07/2014)
3,1217097,5753571,MUT102,LATO SENSU,CANTO II,ESCOLA DE MÚSICA,CONSOLIDADA,26,234567T2345
4,1217097,5753068,MUT102,LATO SENSU,CANTO II,ESCOLA DE MÚSICA,CONSOLIDADA,26,234567T2345


In [81]:
colunas = ["id_turma","id_docente_interno","codigo","nivel_ensino","nome","unidade_responsavel",
          "situacao_turma","capacidade_aluno","descricao_horario"]

merge_turmas_disciplinas6 = pd.merge(turmas6, disciplinas, on='id_componente')[colunas]
merge_turmas_disciplinas6.dropna(inplace=True)
merge_turmas_disciplinas6.head()

,id_turma,id_docente_interno,codigo,nivel_ensino,nome,unidade_responsavel,situacao_turma,capacidade_aluno,descricao_horario
0,1205342,5752548.0,GPU0009,LATO SENSU,INOVAÇÃO NA GESTÃO DE POLÍTICAS PÚBLICAS,PROGRAMA DE PÓS-GRADUAÇÃO EM GESTÃO PÚBLICA,CONSOLIDADA,30.0,7M2345 7T3456 6N1234 (31/01/2014 - 27/03/2014)
1,1205342,21790.0,MUT311,LATO SENSU,APRECIAÇÃO MUSICAL II,ESCOLA DE MÚSICA,CONSOLIDADA,30.0,7M2345 7T3456 6N1234 (31/01/2014 - 27/03/2014)
2,1205343,25444.0,MUT491,LATO SENSU,ARRANJOS II,ESCOLA DE MÚSICA,CONSOLIDADA,30.0,7M2345 7T3456 6N1234
4,1207246,23859.0,MUT102,LATO SENSU,CANTO II,ESCOLA DE MÚSICA,CONSOLIDADA,33.0,7M2345 7T3456
6,1207248,26133.0,MUT102,LATO SENSU,CANTO II,ESCOLA DE MÚSICA,CONSOLIDADA,33.0,7M2345 7T3456


In [82]:
# filtrando as turmas com código IMD
turmas_imd3 = merge_turmas_disciplinas3[merge_turmas_disciplinas3.codigo.str.contains("IMD")]

# utilizar apenas disciplinas com código "CONSOLIDADA"
turmas_imd3 = turmas_imd3[turmas_imd3.situacao_turma == 'CONSOLIDADA']

# Adaptar o nome da coluna id_docente_interno para id_servidor
turmas_imd3.rename(columns={'id_docente_interno': 'id_servidor'}, inplace=True)

turmas_imd3.capacidade_aluno = turmas_imd3.capacidade_aluno.astype(np.int64)

# Adaptar os tipos de dados em turmas_imd para o merge com o dataframe docentes
turmas_imd3.id_servidor = turmas_imd3.id_servidor.astype(np.int64)
turmas_imd3.head()

,id_turma,id_servidor,codigo,nivel_ensino,nome,unidade_responsavel,situacao_turma,capacidade_aluno,descricao_horario
3564,57565129,3588433,IMD0029,GRADUAÇÃO,ESTRUTURA DE DADOS BÁSICAS I,INSTITUTO METROPOLE DIGITAL,CONSOLIDADA,23,45T34
3565,57565340,5757777,IMD0029,GRADUAÇÃO,ESTRUTURA DE DADOS BÁSICAS I,INSTITUTO METROPOLE DIGITAL,CONSOLIDADA,40,35N34
3566,57565345,5755039,IMD0029,GRADUAÇÃO,ESTRUTURA DE DADOS BÁSICAS I,INSTITUTO METROPOLE DIGITAL,CONSOLIDADA,40,345N12
3567,57565346,5757116,IMD0029,GRADUAÇÃO,ESTRUTURA DE DADOS BÁSICAS I,INSTITUTO METROPOLE DIGITAL,CONSOLIDADA,35,2N1234
3601,57565158,5756302,IMD0030,GRADUAÇÃO,LINGUAGEM DE PROGRAMAÇÃO I,INSTITUTO METROPOLE DIGITAL,CONSOLIDADA,30,5T12


In [83]:
# filtrando as turmas com código IMD
turmas_imd4 = merge_turmas_disciplinas4[merge_turmas_disciplinas4.codigo.str.contains("IMD")]

# utilizar apenas disciplinas com código "CONSOLIDADA"
turmas_imd4 = turmas_imd4[turmas_imd4.situacao_turma == 'CONSOLIDADA']

# Adaptar o nome da coluna id_docente_interno para id_servidor
turmas_imd4.rename(columns={'id_docente_interno': 'id_servidor'}, inplace=True)

turmas_imd4.capacidade_aluno = turmas_imd4.capacidade_aluno.astype(np.int64)

# Adaptar os tipos de dados em turmas_imd para o merge com o dataframe docentes
turmas_imd4.id_servidor = turmas_imd4.id_servidor.astype(np.int64)
turmas_imd4.head()

,id_turma,id_servidor,codigo,nivel_ensino,nome,unidade_responsavel,situacao_turma,capacidade_aluno,descricao_horario
3564,1240956,24826,IMD0029,GRADUAÇÃO,ESTRUTURA DE DADOS BÁSICAS I,INSTITUTO METROPOLE DIGITAL,CONSOLIDADA,6,24T12 (02/02/2015 - 20/06/2015)
3565,1241094,5756804,IMD0029,GRADUAÇÃO,ESTRUTURA DE DADOS BÁSICAS I,INSTITUTO METROPOLE DIGITAL,CONSOLIDADA,40,3T1234
3566,1241099,5756052,IMD0029,GRADUAÇÃO,ESTRUTURA DE DADOS BÁSICAS I,INSTITUTO METROPOLE DIGITAL,CONSOLIDADA,60,5N1234
3567,1241100,5753688,IMD0029,GRADUAÇÃO,ESTRUTURA DE DADOS BÁSICAS I,INSTITUTO METROPOLE DIGITAL,CONSOLIDADA,60,3N1234
3601,1240959,16781,IMD0030,GRADUAÇÃO,LINGUAGEM DE PROGRAMAÇÃO I,INSTITUTO METROPOLE DIGITAL,CONSOLIDADA,6,24T12 (02/02/2015 - 20/06/2015)


In [84]:
# filtrando as turmas com código IMD
turmas_imd5 = merge_turmas_disciplinas5[merge_turmas_disciplinas5.codigo.str.contains("IMD")]

# utilizar apenas disciplinas com código "CONSOLIDADA"
turmas_imd5 = turmas_imd5[turmas_imd5.situacao_turma == 'CONSOLIDADA']

# Adaptar o nome da coluna id_docente_interno para id_servidor
turmas_imd5.rename(columns={'id_docente_interno': 'id_servidor'}, inplace=True)

turmas_imd5.capacidade_aluno = turmas_imd5.capacidade_aluno.astype(np.int64)

# Adaptar os tipos de dados em turmas_imd para o merge com o dataframe docentes
turmas_imd5.id_servidor = turmas_imd5.id_servidor.astype(np.int64)
turmas_imd5.head()

,id_turma,id_servidor,codigo,nivel_ensino,nome,unidade_responsavel,situacao_turma,capacidade_aluno,descricao_horario
3564,1230157,22903,IMD0029,GRADUAÇÃO,ESTRUTURA DE DADOS BÁSICAS I,INSTITUTO METROPOLE DIGITAL,CONSOLIDADA,22,4T2345
3565,1230297,5756039,IMD0029,GRADUAÇÃO,ESTRUTURA DE DADOS BÁSICAS I,INSTITUTO METROPOLE DIGITAL,CONSOLIDADA,27,46N12
3566,1230302,22993,IMD0029,GRADUAÇÃO,ESTRUTURA DE DADOS BÁSICAS I,INSTITUTO METROPOLE DIGITAL,CONSOLIDADA,15,35N12
3567,1230303,5755436,IMD0029,GRADUAÇÃO,ESTRUTURA DE DADOS BÁSICAS I,INSTITUTO METROPOLE DIGITAL,CONSOLIDADA,55,5N34 6N12
3568,1230304,25223,IMD0029,GRADUAÇÃO,ESTRUTURA DE DADOS BÁSICAS I,INSTITUTO METROPOLE DIGITAL,CONSOLIDADA,29,24T34


In [85]:
# filtrando as turmas com código IMD
turmas_imd6 = merge_turmas_disciplinas6[merge_turmas_disciplinas6.codigo.str.contains("IMD")]

# utilizar apenas disciplinas com código "CONSOLIDADA"
turmas_imd6 = turmas_imd6[turmas_imd6.situacao_turma == 'CONSOLIDADA']

# Adaptar o nome da coluna id_docente_interno para id_servidor
turmas_imd6.rename(columns={'id_docente_interno': 'id_servidor'}, inplace=True)

turmas_imd6.capacidade_aluno = turmas_imd6.capacidade_aluno.astype(np.int64)

# Adaptar os tipos de dados em turmas_imd para o merge com o dataframe docentes
turmas_imd6.id_servidor = turmas_imd6.id_servidor.astype(np.int64)
turmas_imd6.head()

,id_turma,id_servidor,codigo,nivel_ensino,nome,unidade_responsavel,situacao_turma,capacidade_aluno,descricao_horario
3565,1220343,5755421,IMD0029,GRADUAÇÃO,ESTRUTURA DE DADOS BÁSICAS I,INSTITUTO METROPOLE DIGITAL,CONSOLIDADA,35,2M56 35T12 (27/01/2014 - 14/06/2014)
3566,1220344,5755421,IMD0029,GRADUAÇÃO,ESTRUTURA DE DADOS BÁSICAS I,INSTITUTO METROPOLE DIGITAL,CONSOLIDADA,35,6M34 35T12 (27/01/2014 - 14/06/2014)
3567,1220345,5752964,IMD0029,GRADUAÇÃO,ESTRUTURA DE DADOS BÁSICAS I,INSTITUTO METROPOLE DIGITAL,CONSOLIDADA,35,6M56 35T12 (27/01/2014 - 14/06/2014)
3568,1220345,5756271,IMD0029,GRADUAÇÃO,ESTRUTURA DE DADOS BÁSICAS I,INSTITUTO METROPOLE DIGITAL,CONSOLIDADA,35,6M56 35T12 (27/01/2014 - 14/06/2014)
3601,1220171,5755958,IMD0030,GRADUAÇÃO,LINGUAGEM DE PROGRAMAÇÃO I,INSTITUTO METROPOLE DIGITAL,CONSOLIDADA,36,35M56 4T12 (27/01/2014 - 14/06/2014)


In [86]:
# merge os dataframes  turmas_imd e docentes utilizando como base a coluna id_servidor

colunas = ["id_turma", "codigo", "nivel_ensino", "nome_y",
          "nome_x","capacidade_aluno", "descricao_horario"]
turmas_imd_nomesprof3 = pd.merge(turmas_imd3, docentes, on='id_servidor')[colunas]
lista_prof3 = turmas_imd_nomesprof3.nome_y.unique()

In [87]:
# merge os dataframes  turmas_imd e docentes utilizando como base a coluna id_servidor

turmas_imd_nomesprof4 = pd.merge(turmas_imd4, docentes, on='id_servidor')[colunas]
lista_prof4 = turmas_imd_nomesprof4.nome_y.unique()

In [88]:
# merge os dataframes  turmas_imd e docentes utilizando como base a coluna id_servidor

turmas_imd_nomesprof5 = pd.merge(turmas_imd5, docentes, on='id_servidor')[colunas]
lista_prof5 = turmas_imd_nomesprof5.nome_y.unique()

In [89]:
# merge os dataframes  turmas_imd e docentes utilizando como base a coluna id_servidor

turmas_imd_nomesprof6 = pd.merge(turmas_imd6, docentes, on='id_servidor')[colunas]
lista_prof6 = turmas_imd_nomesprof6.nome_y.unique()

In [91]:
lista_prof3.sort()

lista_prof4.sort()

lista_prof5.sort()

lista_prof6.sort()

In [92]:
avaliacao = pd.read_csv("avaliacaodocencia.csv", sep=';')
av_imd3 = avaliacao[avaliacao['nome_docente'].isin(lista_prof3)]
av_imd4 = avaliacao[avaliacao['nome_docente'].isin(lista_prof4)]
av_imd5 = avaliacao[avaliacao['nome_docente'].isin(lista_prof5)]
av_imd6 = avaliacao[avaliacao['nome_docente'].isin(lista_prof6)]

In [93]:
av_imd_nomesprof3 = pd.merge(av_imd3, turmas_imd3, on='id_turma')

av_imd_nomesprof4 = pd.merge(av_imd4, turmas_imd4, on='id_turma')

av_imd_nomesprof5 = pd.merge(av_imd5, turmas_imd5, on='id_turma')

av_imd_nomesprof6 = pd.merge(av_imd6, turmas_imd6, on='id_turma')

In [94]:
#Concatena os semestres
av_imd_nomesprof_all = pd.concat([av_imd_nomesprof, av_imd_nomesprof2, av_imd_nomesprof3, av_imd_nomesprof4,
                                 av_imd_nomesprof5, av_imd_nomesprof6])
#Concatena a lista dos professores
lista_prof_all = np.concatenate([lista_prof, lista_prof2, lista_prof3, lista_prof4, lista_prof5, lista_prof6])
lista_prof_all.sort()

In [96]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual

def evolucao(x):
    dados_prof = av_imd_nomesprof_all.loc[av_imd_nomesprof_all['nome_docente'] == x]
    return (dados_prof)

interact(evolucao, x=lista_prof_all,);

interactive(children=(Dropdown(description='x', options=('ADA CRISTINA SCUDELARI', 'ADELENA GONCALVES MAIA', 'ADILSON DE LIMA TAVARES', 'ADJA FERREIRA DE ANDRADE', 'ADRIANA ROSA CARVALHO', 'ADRIANO DOS SANTOS', 'ALEX JOSE SOUZA DOS SANTOS', 'ALEX JOSE SOUZA DOS SANTOS', 'ALEXANDRE FADIGAS DE SOUZA', 'ALEXANDRO TEIXEIRA GOMES', 'ALLAN DE MEDEIROS MARTINS', 'ALUIZIO FERREIRA DA ROCHA NETO', 'ALYSON MATHEUS DE CARVALHO SOUZA', 'AMANDA BORGES DE ALBUQUERQUE ASSUNCAO', 'ANA CRISTINA ARAUJO DE ANDRADE GALVAO', 'ANA KATHERINE DA SILVEIRA GONCALVES DE OLIVEIRA', 'ANA PATRICIA DE QUEIROZ MEDEIROS DANTAS', 'ANA PAULA BARRETO GOMES', 'ANA RAFAELA LUZ DE AQUINO MARTINS', 'ANA RAQUEL RODRIGUES LINDQUIST', 'ANDRE FERRER PINTO MARTINS', 'ANDRE LUIS CABRAL DE LOURENCO', 'ANDRE LUIS DORINI', 'ANDRE LUIS LOPES MORIYAMA', 'ANDRE LUIS PINTO SOARES', 'ANDRE LUIZ DA SILVA SOLINO', 'ANDRE LUIZ MARINHO FALCAO GONDIM', 'ANDRE MAURICIO CUNHA CAMPOS', 'ANNE CHRISTINE DAMASIO', 'ANNE CHRISTINE DAMASIO', 'ANNE MAGALY DE PAULA CANUTO', 'ANTONIO DE LISBOA LOPES COSTA', 'ANTONIO DE LISBOA LOPES COSTA', 'ANTONIO IGOR SILVA DE OLIVEIRA', 'ANTONIO RICARDO CALAZANS DUARTE', 'ANTONIO WALLACE ANTUNES SOARES', 'ATHANASIOS TSOUANAS', 'AUGUSTO JOSE VENANCIO NETO', 'AUZELIVIA PASTORA REGO MEDEIROS FALCAO', 'BONIEK CASTILLO DUTRA BORGES', 'BONIEK CASTILLO DUTRA BORGES', 'BRUNA LEAL LIMA MACIEL', 'BRUNO CESAR DE VASCONCELOS GURGEL', 'BRUNO SANTANA DA SILVA', 'CANDIDA MARIA BEZERRA DANTAS', 'CANDIDA MARIA BEZERRA DANTAS', 'CARLA ALMEIDA VIVACQUA', 'CARLOS ALBERTO ALMEIDA DE ARAUJO', 'CARLOS ALBERTO PASKOCIMAS', 'CARLOS AUGUSTO PROLO', 'CARLOS JOSE WANDERLEY FERREIRA', 'CARLOS NECO DA SILVA JUNIOR', 'CARLSON PEREIRA DE SOUZA', 'CAROLINE ADDISON CARVALHO XAVIER DE MEDEIROS', 'CAROLINE ADDISON CARVALHO XAVIER DE MEDEIROS', 'CAROLINE THENNECY DE MEDEIROS ROCHA', 'CATARINA DE OLIVEIRA SOUSA', 'CELSO LUIZ SOUZA DE OLIVEIRA', 'CESAR RENNO COSTA', 'CHARLES ANDRYE GALVAO MADEIRA', 'CICERO FLAVIO SOARES ARAGAO', 'CICLAMIO LEITE BARRETO', 'CLARA MARIA MELO DOS SANTOS', 'CLEONICE ANDREA ALVES CAVALCANTE', 'DAMIANA CLEUMA DE MEDEIROS', 'DANIEL MARQUES DE ALMEIDA PESSOA', 'DANIELLE SOARES BEZERRA', 'DANILO CURVELO DE SOUZA', 'DAVID MENDES', 'DENNYS LEITE MAIA', 'DEUSIMAR FREIRE BRASIL', 'DIEGO ANDRES LAPLAGNE', 'DIEGO DE SOUSA DANTAS', 'DINARA LESLYE MACEDO E SILVA CALAZANS', 'DIOGENES FELIX DA SILVA COSTA', 'DIOMADSON RODRIGUES BELFORT', 'DJALMA RIBEIRO DA SILVA', 'EDGARD DE FARIA CORREA', 'EDMILSON JOVINO DE OLIVEIRA', 'EDMILSON PINTO ALBUQUERQUE', 'EDNA MARIA RANGEL DE SA', 'EDUARDO NOGUEIRA CUNHA', 'EIJI ADACHI MEDEIROS BARBOSA', 'ELAINE CRISTINA GAVIOLI', 'ELIZA MARIA XAVIER FREIRE', 'ENIO CAMPOS AMICO', 'ERIVAN FERREIRA BORGES', 'EULER MACIEL DANTAS', 'EULER MACIEL DANTAS', 'EUZEBIO GUIMARAES BARBOSA', 'EUZEBIO GUIMARAES BARBOSA', 'EVERALDO SILVINO DOS SANTOS', 'EVERTON RANIELLY DE SOUSA CAVALCANTE', 'EZEQUIAS OLIVEIRA LIRA', 'FABIANA BARBOSA GONCALVES', 'FABIANA CRISTINA MENDONCA DE ARAUJO', 'FABIANA LIMA BEZERRA', 'FABIANA TRISTAO DE SANTANA', 'FABIO DE ALMEIDA VIEIRA', 'FABIO MAFRA BORGES', 'FABIO ROBERTO DAMETTO', 'FABIO SPEROTTO BEMFICA', 'FABRICIO PITOMBO LEITE', 'FELIPE BOHN', 'FERNANDA DA FONSECA FREITAS', 'FERNANDA DINIZ DE SA', 'FERNANDO HENRIQUE ANDRADE NOGUEIRA', 'FLAVIO EDUARDO FALCAO LEMOS', 'FLAVIO MEDEIROS JUNIOR', 'FRANCISCA IDANESIA DA SILVA', 'FRANCISCO NABUCO DE ALMEIDA BARRETO NETO', 'GERLANE COELHO BERNARDO GUERRA', 'GIBEON SOARES DE AQUINO JUNIOR', 'GILMAR AMORIM DE SOUSA', 'GILSON DE VASCONCELOS TORRES', 'GISELLE COSTA DE SOUSA', 'GISLENE MARIA DA SILVA GANADE', 'GLEYDSON PINHEIRO ALBANO', 'GUILHERME AUGUSTO DE FREITAS FREGONEZI', 'GUSTAVO AUGUSTO SEABRA BARBOSA', 'GUSTAVO AUGUSTO SEABRA BARBOSA', 'GUSTAVO GIRAO BARRETO DA SILVA', 'HAILSON ALVES FERREIRA PRESTON', 'HECTOR LENY CARRION SALAZAR', 'HELCIO PACHECO DE MEDEIROS', 'HELDER ALEXANDRE MEDEIROS DE MACEDO', 'HERNANI DE PAIVA GADELHA JUNIOR', 'HUGO ALEXANDRE DE OLIVEIRA ROCHA', 'IDIVALDO ANTONIO MICAL

In [97]:
% matplotlib inline
from ipywidgets import interactive
import matplotlib.pyplot as plt
import numpy as np

def evolucaoGrafico(x):
    dados_prof_all = av_imd_nomesprof_all.loc[av_imd_nomesprof_all['nome_docente'] == x]
    plt.barh(dados_prof_all['nome'], dados_prof_all['postura_profissional_media'], 0.3)
    plt.xlabel("Notas")
    plt.ylabel("Disciplinas")
    plt.title("Titulo")
    plt.show()
    
interactive_plot = interact(evolucaoGrafico, x=lista_prof_all,)
interactive_plot

interactive(children=(Dropdown(description='x', options=('ADA CRISTINA SCUDELARI', 'ADELENA GONCALVES MAIA', 'ADILSON DE LIMA TAVARES', 'ADJA FERREIRA DE ANDRADE', 'ADRIANA ROSA CARVALHO', 'ADRIANO DOS SANTOS', 'ALEX JOSE SOUZA DOS SANTOS', 'ALEX JOSE SOUZA DOS SANTOS', 'ALEXANDRE FADIGAS DE SOUZA', 'ALEXANDRO TEIXEIRA GOMES', 'ALLAN DE MEDEIROS MARTINS', 'ALUIZIO FERREIRA DA ROCHA NETO', 'ALYSON MATHEUS DE CARVALHO SOUZA', 'AMANDA BORGES DE ALBUQUERQUE ASSUNCAO', 'ANA CRISTINA ARAUJO DE ANDRADE GALVAO', 'ANA KATHERINE DA SILVEIRA GONCALVES DE OLIVEIRA', 'ANA PATRICIA DE QUEIROZ MEDEIROS DANTAS', 'ANA PAULA BARRETO GOMES', 'ANA RAFAELA LUZ DE AQUINO MARTINS', 'ANA RAQUEL RODRIGUES LINDQUIST', 'ANDRE FERRER PINTO MARTINS', 'ANDRE LUIS CABRAL DE LOURENCO', 'ANDRE LUIS DORINI', 'ANDRE LUIS LOPES MORIYAMA', 'ANDRE LUIS PINTO SOARES', 'ANDRE LUIZ DA SILVA SOLINO', 'ANDRE LUIZ MARINHO FALCAO GONDIM', 'ANDRE MAURICIO CUNHA CAMPOS', 'ANNE CHRISTINE DAMASIO', 'ANNE CHRISTINE DAMASIO', 'ANNE MAGALY DE PAULA CANUTO', 'ANTONIO DE LISBOA LOPES COSTA', 'ANTONIO DE LISBOA LOPES COSTA', 'ANTONIO IGOR SILVA DE OLIVEIRA', 'ANTONIO RICARDO CALAZANS DUARTE', 'ANTONIO WALLACE ANTUNES SOARES', 'ATHANASIOS TSOUANAS', 'AUGUSTO JOSE VENANCIO NETO', 'AUZELIVIA PASTORA REGO MEDEIROS FALCAO', 'BONIEK CASTILLO DUTRA BORGES', 'BONIEK CASTILLO DUTRA BORGES', 'BRUNA LEAL LIMA MACIEL', 'BRUNO CESAR DE VASCONCELOS GURGEL', 'BRUNO SANTANA DA SILVA', 'CANDIDA MARIA BEZERRA DANTAS', 'CANDIDA MARIA BEZERRA DANTAS', 'CARLA ALMEIDA VIVACQUA', 'CARLOS ALBERTO ALMEIDA DE ARAUJO', 'CARLOS ALBERTO PASKOCIMAS', 'CARLOS AUGUSTO PROLO', 'CARLOS JOSE WANDERLEY FERREIRA', 'CARLOS NECO DA SILVA JUNIOR', 'CARLSON PEREIRA DE SOUZA', 'CAROLINE ADDISON CARVALHO XAVIER DE MEDEIROS', 'CAROLINE ADDISON CARVALHO XAVIER DE MEDEIROS', 'CAROLINE THENNECY DE MEDEIROS ROCHA', 'CATARINA DE OLIVEIRA SOUSA', 'CELSO LUIZ SOUZA DE OLIVEIRA', 'CESAR RENNO COSTA', 'CHARLES ANDRYE GALVAO MADEIRA', 'CICERO FLAVIO SOARES ARAGAO', 'CICLAMIO LEITE BARRETO', 'CLARA MARIA MELO DOS SANTOS', 'CLEONICE ANDREA ALVES CAVALCANTE', 'DAMIANA CLEUMA DE MEDEIROS', 'DANIEL MARQUES DE ALMEIDA PESSOA', 'DANIELLE SOARES BEZERRA', 'DANILO CURVELO DE SOUZA', 'DAVID MENDES', 'DENNYS LEITE MAIA', 'DEUSIMAR FREIRE BRASIL', 'DIEGO ANDRES LAPLAGNE', 'DIEGO DE SOUSA DANTAS', 'DINARA LESLYE MACEDO E SILVA CALAZANS', 'DIOGENES FELIX DA SILVA COSTA', 'DIOMADSON RODRIGUES BELFORT', 'DJALMA RIBEIRO DA SILVA', 'EDGARD DE FARIA CORREA', 'EDMILSON JOVINO DE OLIVEIRA', 'EDMILSON PINTO ALBUQUERQUE', 'EDNA MARIA RANGEL DE SA', 'EDUARDO NOGUEIRA CUNHA', 'EIJI ADACHI MEDEIROS BARBOSA', 'ELAINE CRISTINA GAVIOLI', 'ELIZA MARIA XAVIER FREIRE', 'ENIO CAMPOS AMICO', 'ERIVAN FERREIRA BORGES', 'EULER MACIEL DANTAS', 'EULER MACIEL DANTAS', 'EUZEBIO GUIMARAES BARBOSA', 'EUZEBIO GUIMARAES BARBOSA', 'EVERALDO SILVINO DOS SANTOS', 'EVERTON RANIELLY DE SOUSA CAVALCANTE', 'EZEQUIAS OLIVEIRA LIRA', 'FABIANA BARBOSA GONCALVES', 'FABIANA CRISTINA MENDONCA DE ARAUJO', 'FABIANA LIMA BEZERRA', 'FABIANA TRISTAO DE SANTANA', 'FABIO DE ALMEIDA VIEIRA', 'FABIO MAFRA BORGES', 'FABIO ROBERTO DAMETTO', 'FABIO SPEROTTO BEMFICA', 'FABRICIO PITOMBO LEITE', 'FELIPE BOHN', 'FERNANDA DA FONSECA FREITAS', 'FERNANDA DINIZ DE SA', 'FERNANDO HENRIQUE ANDRADE NOGUEIRA', 'FLAVIO EDUARDO FALCAO LEMOS', 'FLAVIO MEDEIROS JUNIOR', 'FRANCISCA IDANESIA DA SILVA', 'FRANCISCO NABUCO DE ALMEIDA BARRETO NETO', 'GERLANE COELHO BERNARDO GUERRA', 'GIBEON SOARES DE AQUINO JUNIOR', 'GILMAR AMORIM DE SOUSA', 'GILSON DE VASCONCELOS TORRES', 'GISELLE COSTA DE SOUSA', 'GISLENE MARIA DA SILVA GANADE', 'GLEYDSON PINHEIRO ALBANO', 'GUILHERME AUGUSTO DE FREITAS FREGONEZI', 'GUSTAVO AUGUSTO SEABRA BARBOSA', 'GUSTAVO AUGUSTO SEABRA BARBOSA', 'GUSTAVO GIRAO BARRETO DA SILVA', 'HAILSON ALVES FERREIRA PRESTON', 'HECTOR LENY CARRION SALAZAR', 'HELCIO PACHECO DE MEDEIROS', 'HELDER ALEXANDRE MEDEIROS DE MACEDO', 'HERNANI DE PAIVA GADELHA JUNIOR', 'HUGO ALEXANDRE DE OLIVEIRA ROCHA', 'IDIVALDO ANTONIO MICAL

<function __main__.evolucaoGrafico>